In [1]:
run FunctionsLoader.ipynb

0.2.0_4
0.2.0_4


In [2]:
class Predictions():
    
    
    def __init__(self, min_days=30):
        self.min_days = min_days
            
        
    # time_window should be in mins (e.g; 30mins)
    # sd_ratio should be between 0 and 1 (e.g; 0.25 for using 1/4 of sd)

    def loadData(self, time_window, sd_ratio, t_hist):
        
        ### read mobility data ###
        ld = LocationData()
        ld.readLocationData(precision_value=4) # read file
        ld.filterUser(self.min_days) #filter users with min 30 days
        #print len(ld.users)
        ld.convertToEqualTimeSeries(time_window * 60 * 1000) # convert to time series of 30mins
        dt = ld.createUserDailyTrajectoryLists() # create daily trajectories of users 

        ### read mood data ###
        pd = PhqData()
        pd.readPhqData()
        pd.computeScores(ld.users, self.min_days)
        pd.computeDivergence()
        pd.convertTo2Labels(sd_ratio=sd_ratio) 
        #len(list(set(m[0] for m in pd.phq_scores)))

        ### merge data ###
        mlm = PhqLocMerge()
        mlm.mergeData(dt, pd.phq_scores, t_hist)
        
        ### set data as a class variable ###
        self.data = mlm.data

    
    
    def prepareInput(self):
    
        ###  transform mobility data ### 
        users = list(set([m[0] for m in self.data]))
        dt2 = approachAvg2(lm_data=self.data, users=users) # Computing displacement changes
        dt3 = approachAvg3(lm_data=self.data, users=users) # Computing displacement [normalised]
        dt4 = approachAvg4(lm_data=self.data, users=users, topn=20) # Computing displacement [normalised]

        ###  combinations of data for using different combinations of features ### 
        self.data1 = [dt2] 
        self.data2 = [dt3] 
        self.data3 = [dt4] 
        self.data4 = [dt2,dt3] 
        self.data5 = [dt2,dt4] 
        self.data6 = [dt3,dt4] 
        self.data7 = [dt2,dt3,dt4]
    
    
    def computePredictions(self, is_generic=False, predict_raw_score=False, layers=5, act_func=''):

        ### filter users ### 
        filtered_users = []
        users = list(set([m[0] for m in self.data]))
        for u in users:
            ud = [d for d in self.data1[0] if d[0]==u]
            if len(ud) > self.min_days:
                filtered_users.append(u)
        print 'Number of users', len(filtered_users)
        
        ### computing prdiction accuracy ### 
        if is_generic:
            model1 = ClassifierMultiDimenHelper.compute_svm_accuracy(self.data1, iteration=20, users=filtered_users, min_days=self.min_days) 
            model2 = ClassifierMultiDimenHelper.compute_svm_accuracy(self.data2, iteration=20, users=filtered_users, min_days=self.min_days) 
            model3 = ClassifierMultiDimenHelper.compute_svm_accuracy(self.data3, iteration=20, users=filtered_users, min_days=self.min_days) 
            model4 = ClassifierMultiDimenHelper.compute_svm_accuracy(self.data4, iteration=20, users=filtered_users, min_days=self.min_days) 
            model5 = ClassifierMultiDimenHelper.compute_svm_accuracy(self.data5, iteration=20, users=filtered_users, min_days=self.min_days) 
            model6 = ClassifierMultiDimenHelper.compute_svm_accuracy(self.data6, iteration=20, users=filtered_users, min_days=self.min_days) 
            model7 = ClassifierMultiDimenHelper.compute_svm_accuracy(self.data7, iteration=20, users=filtered_users, min_days=self.min_days) 

        else:
            model1 = ClassifierPersonalisedHelperRawScores.compute_svm_accuracy(self.data1, iteration=20, users=filtered_users, min_days=self.min_days, predict_raw_score=predict_raw_score, layers=layers, act_func=act_func) 
            model2 = ClassifierPersonalisedHelperRawScores.compute_svm_accuracy(self.data2, iteration=20, users=filtered_users, min_days=self.min_days, predict_raw_score=predict_raw_score, layers=layers, act_func=act_func) 
            model3 = ClassifierPersonalisedHelperRawScores.compute_svm_accuracy(self.data3, iteration=20, users=filtered_users, min_days=self.min_days, predict_raw_score=predict_raw_score, layers=layers, act_func=act_func) 
            model4 = ClassifierPersonalisedHelperRawScores.compute_svm_accuracy(self.data4, iteration=20, users=filtered_users, min_days=self.min_days, predict_raw_score=predict_raw_score, layers=layers, act_func=act_func) 
            model5 = ClassifierPersonalisedHelperRawScores.compute_svm_accuracy(self.data5, iteration=20, users=filtered_users, min_days=self.min_days, predict_raw_score=predict_raw_score, layers=layers, act_func=act_func) 
            model6 = ClassifierPersonalisedHelperRawScores.compute_svm_accuracy(self.data6, iteration=20, users=filtered_users, min_days=self.min_days, predict_raw_score=predict_raw_score, layers=layers, act_func=act_func) 
            model7 = ClassifierPersonalisedHelperRawScores.compute_svm_accuracy(self.data7, iteration=20, users=filtered_users, min_days=self.min_days, predict_raw_score=predict_raw_score, layers=layers, act_func=act_func) 

        self.models = [model1, model2, model3, model4, model5, model6, model7]

        
    def plotPredictions(self, include_mt=False, t_hist=14, file_path=''):
        model_names = ['IR 1','IR 2','IR 3','IR 1+2','IR 1+3','IR 2+3','IR 1+2+3']
        label_dict = {0:'R2', 1:'MSE'}
        Plots.multiModelRawScores(models=p.models, model_names=model_names, label_dict=label_dict, include_mt=include_mt, t_hist=t_hist, file_path=file_path)
        
        


In [ ]:
all_models = [] # store models of all iterations (for debugging)
for layers in [5,7,9]:
    for time_window in [10,30,60]:
        for t_hist in [1,7,14]:
            print 'Iteration values:', time_window, t_hist
            p = Predictions(30)
            p.loadData(time_window=time_window, sd_ratio=0.25, t_hist=t_hist)
            p.prepareInput()
            p.computePredictions(is_generic=False, predict_raw_score=True, layers=layers, act_func='elu')
            all_models = p.models
            file_path = 'plots/regression/elu/raw_score_prediction_time_window_' + str(time_window) + '_t_hist_' + str(t_hist)  + '_ae_' + str(layers)  + '.pdf'
            p.plotPredictions(include_mt=True, t_hist=t_hist, file_path=file_path)

Iteration values: 10 1
Initial user count 6705
User count 5616
Min time ratio 0.500023134225
Number of users 44
Pos count: 900
Neg count: 1489
Number of users 23
Ecoded X data
Performance computed:  -0.177184762236 11.2091303124
Performance computed:  -0.121382448311 8.26097636524
Performance computed:  -0.276361110404 11.5024492164
Performance computed:  -0.31623650297 11.22748356
Performance computed:  -0.0403786122814 11.0731052002
Performance computed:  -0.126484045807 14.4204835505
Performance computed:  -0.150958752205 13.5200849689
Performance computed:  -0.183104227223 8.55325060761
Performance computed:  -0.171913099614 12.7757603843
Performance computed:  -0.0492122282276 8.55864337081
Performance computed:  -0.117767974538 9.40754022641
Performance computed:  -0.255158677312 10.2069648037
Performance computed:  -0.213230107818 11.0867779215
Performance computed:  -0.161942473817 11.1218694251
Performance computed:  -0.169713827288 10.5877011597
Performance computed:  -0.1019

Performance computed:  -1.15477614463 17.3850565956
Performance computed:  -0.861559197146 17.9251437158
Performance computed:  -0.263449779311 14.796375451
Performance computed:  -0.780986457739 10.7065520216
Performance computed:  -0.129738237036 11.4315308811
Performance computed:  -0.0500131530474 12.4315447965
Performance computed:  -0.108114855541 14.3519641896
Performance computed:  -0.181385893214 12.1168826611
Performance computed:  -0.511069457911 12.8298954457
Performance computed:  -0.0699376433022 8.06843645629
Performance computed:  -0.526740941646 12.3732104667
Performance computed:  -0.557896948312 15.2083235738
Performance computed:  -0.217898540778 10.1902898594
Performance computed:  -0.262315484432 10.2570197122
Performance computed:  -0.0587664386857 9.33891503539
Performance computed:  -0.336708286356 13.4291859697
Performance computed:  -0.182504971052 13.658771057
Performance computed:  0.0391186470022 9.77794917315
Performance computed:  -0.0691013558126 11.048

Performance computed:  -0.370808057836 8.55002739588
Performance computed:  0.170945956327 4.16337157935
Performance computed:  0.0515965105437 5.57808758482
Performance computed:  -0.238325393668 9.11250687141
Performance computed:  -0.587020180087 9.4210055868
Performance computed:  -0.448971235525 9.01346008543
Performance computed:  -0.0631850953771 4.97985130654
Performance computed:  -0.210419858104 6.56880058303
Performance computed:  -0.145767716204 7.41898883179
Performance computed:  -0.536431838082 6.67673319692
Performance computed:  -0.508393024825 6.88350027095
Performance computed:  -0.155140830777 7.26739057223
Performance computed:  0.0287715549319 6.08963912301
Performance computed:  -0.588915291086 10.7334582967
Performance computed:  -0.491754707639 9.18017286955
Performance computed:  -0.130406568667 7.19174857664
Performance computed:  -0.721525841776 7.0621354899
Performance computed:  -0.262841267961 2.43671073186
Performance computed:  0.0399670400265 1.1138098

Performance computed:  -0.0965262737335 12.8715380134
Performance computed:  -0.299955991633 4.20736647863
Performance computed:  -1.75800318925 2.8943790025
Performance computed:  -0.810209455861 2.13576178679
Performance computed:  -0.700298663326 4.24767948596
Performance computed:  -0.212962381411 3.99890443909
Performance computed:  -0.916991041868 4.18878232307
Performance computed:  -0.40743294751 3.22876255879
Performance computed:  -0.870191349123 4.49450307673
Performance computed:  -0.693676438613 1.43201115993
Performance computed:  -0.266039086176 4.26526793325
Performance computed:  -0.740930674909 3.38427757163
Performance computed:  -0.357043301255 3.80878110361
Performance computed:  -0.658090689176 4.02675055043
Performance computed:  -0.12504680533 2.92381466472
Performance computed:  -0.0245048641636 2.36081934038
Performance computed:  -2.30245284209 2.84977553083
Performance computed:  -0.4115920652 2.96119521563
Performance computed:  -0.0129332352396 1.120593452

Performance computed:  0.274838681759 7.49391175071
Performance computed:  -0.121433745294 11.5145424175
Performance computed:  -0.436345610523 9.35521530883
Performance computed:  -0.464566577264 15.9626026184
Performance computed:  -0.359548119596 13.2339478858
Performance computed:  -7.98059213571 1.10965868835
Performance computed:  -0.0790858267203 2.67450871599
Performance computed:  -1.20055955017 4.98666332766
Performance computed:  0.0698963272674 0.927381972678
Performance computed:  0.280110213467 1.00411259921
Performance computed:  0.0225351837112 3.82061469104
Performance computed:  -0.173483518063 3.32116790675
Performance computed:  -0.0789850620661 1.50450979592
Performance computed:  -0.360427141925 4.25545167234
Performance computed:  -0.109103325576 2.44571901573
Performance computed:  -2.38676964051 2.81952892432
Performance computed:  -0.195257933394 2.22590745465
Performance computed:  -0.261997083013 2.7518082483
Performance computed:  -0.701872127672 3.49270904

...

Performance computed:  0.405652954798 7.05294087008
Performance computed:  -0.152124208681 8.02909080817
Performance computed:  -0.0521985529525 9.45432512491
Performance computed:  -0.1844803181 13.7692959587
Performance computed:  0.125301600406 10.2496324284
Performance computed:  -0.188158757484 13.9206373976
Performance computed:  0.143741544303 10.0579752416
Performance computed:  0.162365065817 9.04097410684
Performance computed:  0.126767302127 9.92356019715
Performance computed:  -0.378330130378 14.1442802461
Performance computed:  0.135512872434 8.50545899835
Performance computed:  -0.100625163479 12.3442494745
Performance computed:  -0.660934078231 2.67942669359
Performance computed:  -1.57897294524 3.72920730584
Performance computed:  -0.179320702239 3.29667268991
Performance computed:  -1.5141666248 2.92543115788
Performance computed:  -0.639197174315 1.99288137755
Performance computed:  -0.554307533478 5.77593764172
Performance computed:  -0.223903535809 3.63365485686
Per

Performance computed:  -3.40589094566 1.50825179293
Performance computed:  -0.0694210224584 1.70402424703
Performance computed:  -0.451456855129 4.09357030978
Performance computed:  -1.16282742718 1.17950745202
Performance computed:  -5.68042052436 1.28947059332
Performance computed:  0.401692782596 3.07147254091
Performance computed:  0.121210374468 5.44776709851
Performance computed:  0.632788499944 2.49547096964
Performance computed:  0.199515608373 4.93055441492
Performance computed:  0.280893924046 4.37040808096
Performance computed:  -0.285196874041 5.5839626103
Performance computed:  0.209382173355 5.54298535007
Performance computed:  -0.138552473878 6.33734504587
Performance computed:  0.205828072148 5.03857369205
Performance computed:  -0.0286061551687 5.11404615626
Performance computed:  -0.204727178631 8.16404602745
Performance computed:  0.224917162059 5.01654908774
Performance computed:  0.171011362323 3.67952048194
Performance computed:  0.00867430691976 4.23479031182
Per

Performance computed:  0.11664388872 7.94703224501
Performance computed:  -0.0649176660833 12.0145332087
Performance computed:  -0.542238150629 13.0383968018
Performance computed:  -0.115042392921 11.3682902738
Performance computed:  0.0563730989419 8.35738230296
Performance computed:  -0.00530422642002 2.79351435133
Performance computed:  -0.132733993884 1.20016616145
Performance computed:  -12.3531425767 2.37839897027
Performance computed:  -0.0544827077739 2.49718968475
Performance computed:  0.43614862873 1.90555948505
Performance computed:  -64.0940031951 2.13825136364
Performance computed:  -0.645270449922 2.08589498025
Performance computed:  0.254047421583 1.2392690928
Performance computed:  -2.53586163232 2.8845268926
Performance computed:  0.466741089863 1.70588572067
Performance computed:  0.466532105453 1.07293546646
Performance computed:  0.00572450979155 2.71488489074
Performance computed:  0.275774917 1.7799278847
Performance computed:  0.169932621848 2.26477152365
Perfor

Performance computed:  0.0214834641934 1.41493962585
Performance computed:  -0.408701076944 3.93788251134
Performance computed:  -1.18898865285 2.54706094104
Performance computed:  -0.628599572184 1.97999716553
Performance computed:  0.0466034551341 3.54290182115
Performance computed:  -0.139442177335 3.3828970018
Performance computed:  -0.777836970864 2.35629160466
Performance computed:  -0.502044683555 4.41518115788
Performance computed:  0.106754629484 1.44874372874
Performance computed:  -1.20173195911 1.72676585973
Performance computed:  -0.221674871108 3.71521612812
Performance computed:  0.228883368884 1.73529765448
Performance computed:  -0.292921613002 3.3538493835
Performance computed:  0.116758493991 3.17634881661
Performance computed:  -0.0209753989646 0.845495252268
Performance computed:  -3.62294821561 0.541350446429
Performance computed:  0.0340107486278 2.2432707979
Performance computed:  -0.275617792303 1.88022374575
Performance computed:  -0.757309708191 1.86635767432

Performance computed:  -0.00221779272705 4.2813196799
Performance computed:  0.117090973014 6.83845287944
Performance computed:  -0.0818090530246 7.31407304243
Performance computed:  -0.0793366521687 6.43799254491
Performance computed:  0.0403336926434 5.53019414275
Performance computed:  -0.0986493066029 7.54845873257
Performance computed:  -0.241886309363 6.73789544435
Performance computed:  -0.0683717430233 1.94738578765
Performance computed:  -0.146688650481 1.02219952796
Performance computed:  -0.0791859741617 1.70471759827
Performance computed:  0.0238446956684 1.82067413308
Performance computed:  -0.289379227398 2.60180964052
Performance computed:  -0.501127705628 2.83301062092
Performance computed:  -0.317905055823 1.39555555556
Performance computed:  -0.176297450554 1.15718996602
Performance computed:  0.115046797816 0.981495098039
Performance computed:  -0.0731644161936 1.6421870519
Performance computed:  -0.0956795381193 1.68043161765
Performance computed:  -0.52525938717 1.

In [4]:
# mn = ['IR 1','IR 2','IR 3','IR 1+2','IR 1+3','IR 2+3','IR 1+2+3']
# ld = {1:'Sensitivity', 0:'Specificity'}
# Plots.multiModel(models=p.models, model_names=mn, label_dict=ld, include_mt=True, t_hist=7, file_path='plots/prediction_plot.png')